In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# download S&P 500 data pricing hystory for 5 years

sp500 = yf.download('^GSPC', period='5y')
sp500.head()

In [ ]:
# sliding mean analysis

sp500['SMA_50'] = sp500['Close'].rolling(window=50).mean()
sp500['SMA_200'] = sp500['Close'].rolling(window=200).mean()

# Plotting the closing price and the moving averages
plt.figure(figsize=(14, 7))
plt.plot(sp500['Close'], label='S&P 500 Close Price', color='blue')
plt.plot(sp500['SMA_50'], label='50-Day SMA', color='orange')
plt.plot(sp500['SMA_200'], label='200-Day SMA', color='red')
plt.title('S&P 500 Closing Price and Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# compute the daily returns
sp500['Daily Return'] = sp500['Close'].pct_change()
print(sp500['Daily Return'].describe())
# Plotting the daily returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Daily Return'], label='Daily Return', color='green')
plt.title('S&P 500 Daily Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute cumuilative returns
sp500['Cumulative Return'] = (1 + sp500['Daily Return']).cumprod()
print(sp500['Cumulative Return'].describe())
# Plotting the cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Cumulative Return'], label='Cumulative Return', color='purple')
plt.title('S&P 500 Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute rolling volatility
sp500['Rolling Volatility'] = sp500['Daily Return'].rolling(window=21).std() * (252 ** 0.5)  # Annualized volatility
print(sp500['Rolling Volatility'].describe())

# Plotting the rolling volatility and MAD
plt.figure(figsize=(14, 7))
plt.plot(sp500['Rolling Volatility'], label='Rolling Volatility (Annualized)', color='brown')
plt.title('S&P 500 Rolling Volatility (Annualized)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()  

In [ ]:
# compute MAD and quantiles
sp500['MAD'] = sp500['Close'].rolling(window=50).apply(lambda x: (x - x.mean()).abs().mean())
print(sp500['MAD'].describe())
# Plotting the Mean Absolute Deviation (MAD)
plt.figure(figsize=(14, 7))
plt.plot(sp500['MAD'], label='Mean Absolute Deviation (MAD)', color='cyan')
plt.title('S&P 500 Mean Absolute Deviation (MAD)')
plt.xlabel('Date')
plt.ylabel('MAD')
plt.legend()
plt.grid()
plt.show()      